### **Foundation**

Use Python to fetch XAUUSDc H1 candles data from MT5. UTC-based timezone. Use plotly whitethemed default candlestick chart. Identify the 'trading_daily_session' as from market-open to market-close, which includes pre-market and overnight time. Plot virtical-dashed line for every first open-candle of the 'trading_daily_session'  and the closing-candle of the 'trading_daily_session'. To easily identify the 'trading_daily_session', simply locate the very first candle recorded in pre-market and the last candle recorded at overnight.

```python

# Parameters
symbol = "XAUUSDc"
timeframe = mt5.TIMEFRAME_H1
utc_timezone = pytz.UTC
lookback_days = 10  # fetch last 10 days of H1 candles


Use Python to fetch XAUUSDc H1 candles data from MT5. UTC based timezone. Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight session.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight session.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight sessions. Make a developing RSI indicator that starts calculating from the start of the trading day and resets at the close of the trading day. PLot the tivk-volume bars on separate pane, plot the RSI indicaor in seeparate pane.